In [ ]:
%run dqn.py plot

In [1]:
%run dqn.py

[2017-08-19 06:25:00,019] Making new env: Pong-v0


Input (32, 4, 84, 84)
Feat torch.Size([32, 64, 7, 7])
Model loaded from save/cp49950000.torchmodel
, Step 49950001, Eps 0.099999


In [ ]:
# plot(status['duration_history'])

In [2]:
import matplotlib.pyplot as plt
from matplotlib import gridspec

In [3]:
# State to image
def state_to_image(s, decay=0.5):
    sim = s[-1].astype(np.float)
    coeff = decay
    coeff_sum = 1.0
    for i in range(s.shape[0]-1, -1, -1):
        sim += s[i].astype(np.float) * coeff
        coeff_sum += coeff
        coeff *= decay
    sim /= coeff_sum
    sim = sim.astype(np.uint8)
    return sim

In [4]:
-

In [5]:
pol = Policy(model, ACTION_NUM, 0, 0, 0)

In [15]:
env1 = Env(preproc, 4, FRAMES_PER_STATE)

[2017-08-19 06:50:18,550] Making new env: Pong-v0


In [16]:
env_ = env1 # try to evaluate every step
s  = env_.reset()
stop = False
t = 0
MAX_DEMO_STEPS = 10000
accumulated_r = 0
while not stop:
    v = pol.get_action_val(s)
    act_id = np.argmax(v)
    draw_state_evaluation(s, v, act_id, t, 'out/f')
    act_button = ACTION_BUTTON_MAP[act_id]
    s, r, term = env_.step(act_button)
    accumulated_r += r
    t += 1 
    # print t, v, act_id
    if r != 0:
        print "Step: {}, Accumulated Rewards {}".format(t, accumulated_r)
    if t >= MAX_DEMO_STEPS or term:
        stop = True


T=89 Step: 90, Accumulated Rewards 1.0             
T=142                              Step: 143, Accumulated Rewards 2.0
T=174 Step: 175, Accumulated Rewards 3.0
T=195 Step: 196, Accumulated Rewards 2.0
T=206 Step: 207, Accumulated Rewards 1.0
T=238 Step: 239, Accumulated Rewards 0.0
T=270 Step: 271, Accumulated Rewards -1.0
T=344 Step: 345, Accumulated Rewards -2.0       
T=375 Step: 376, Accumulated Rewards -3.0
T=418 Step: 419, Accumulated Rewards -2.0
T=459 Step: 460, Accumulated Rewards -3.0
T=501                        Step: 502, Accumulated Rewards -2.0
T=594 Step: 595, Accumulated Rewards -1.0                               
T=655                                          Step: 656, Accumulated Rewards -2.0
T=758                                                             Step: 759, Accumulated Rewards -1.0
T=820 Step: 821, Accumulated Rewards -2.0
T=852                  Step: 853, Accumulated Rewards -3.0
T=1006 Step: 1007, Accumulated Rewards -4.0                              

In [12]:
%%bash
cd out
ffmpeg -y -framerate 10 -i f_%05d.png -c:v libx264 -pix_fmt yuv420p out.mp4

ffmpeg version 2.8.11-0ubuntu0.16.04.1 Copyright (c) 2000-2017 the FFmpeg developers
  built with gcc 5.4.0 (Ubuntu 5.4.0-6ubuntu1~16.04.4) 20160609
  configuration: --prefix=/usr --extra-version=0ubuntu0.16.04.1 --build-suffix=-ffmpeg --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --cc=cc --cxx=g++ --enable-gpl --enable-shared --disable-stripping --disable-decoder=libopenjpeg --disable-decoder=libschroedinger --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libopenjpeg --enable-libopus --enable-libpulse --enable-librtmp --enable-libschroedinger --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --e

In [ ]:
tmp_p = list(model.parameters())

In [ ]:
print model.modules

In [ ]:
def draw_conv_parameters(cparam, fname=None):
    plist = list(cparam)
    weights = plist[0].data.cpu().numpy()
    wlim = max(np.abs(weights.max()), np.abs(weights.min()))
    
    nout = weights.shape[0]
    nin = weights.shape[1]
    print "in", nin, "out", nout
    
    gs = gridspec.GridSpec(
        nout, nin, 
        left=0.01, right=0.99, top=0.99, bottom=0.01,
        hspace=0.01, wspace=0.01)
    
    plt.figure(2, figsize=(nin/2, nout/2))
    i = 0
    for r in range(nout):
        row_wextreme = np.max(np.abs(weights[r, ...]))
        for c in range(nin):
            ax = plt.subplot(gs[i])
            ax.imshow(weights[r, c], cmap='gray',
                      interpolation='nearest',
                      vmax=wlim, vmin=-wlim)
            ax.set_xticks([])
            ax.set_yticks([])
            i += 1
        print "row", r, row_wextreme
    
    if fname is not None:
        plt.savefig(fname)
        
    
    

In [ ]:
draw_conv_parameters(model.conv2.parameters(), 'out/p_conv2.png')

In [ ]:
subnets = (model.conv1, model.conv2, model.conv3)
subnetnames = ('conv1', 'conv2', 'conv3')
for sn, fn in zip(subnets, subnetnames):
    plt.clf()
    print "Drawing {} ...".format(fn)
    draw_conv_parameters(sn.parameters(), 'out/{}_weights.png'.format(fn))

In [ ]:
out_parameters = list(model.head[2].parameters())
out_p0 = out_parameters[0].data.cpu().numpy()
out_p1 = out_parameters[1].data.cpu().numpy()

In [ ]:
out_p1.shape

In [ ]:
for out_id in range(3):
    plt.clf()
    plt.imshow(out_p0[out_id].reshape((16, 32)), cmap='gray', 
               interpolation='nearest')
    plt.savefig("out/tmp_output_parameter_{}.png".format(out_id))